In [1]:
#imports
from Bio import Entrez
from Bio import SeqIO
import numpy as np
import time
Entrez.email="email@domain.com" ## email here


In [2]:
#Save the Genbank File format use biopython
def getGenBankFile(accession_number):
    accession=accession_number
    filename=accession+".gb"
    getfile=Entrez.efetch(db="nucleotide",rettype="gb", retmode="text", id=accession)
    writeFile=open(filename,'w')
    writeFile.write(getfile.read())
    getfile.close()
    writeFile.close()
    print("File Saved")
getGenBankFile("NC_001133.9")  
getGenBankFile("NC_047487.1")

File Saved
File Saved


In [3]:
#Extract the list of CDSs from files with gene products 
def extractListofCDS(GB_filename):
    all_entries = {} # all CDS
    all_products={} # all CDS gene names
    file_name=GB_filename
    with open(file_name, 'r') as GBFile:
        GBcds = SeqIO.InsdcIO.GenBankCdsFeatureIterator(GBFile) # used genbank iterator from SeqIO module in Biopython to get CDS
        for cds in GBcds:
            if cds.seq is not None:
                all_entries[cds.id]=cds.seq
                all_products[cds.description.lower().split()[-1]]=cds.id
    return all_entries, all_products

# get common coding sequences from two genbank files randomly
def getcommoncds(filename1,filename2):
    file1_seq, file1_prod=extractListofCDS(filename1)
    file2_seq, file2_prod=extractListofCDS(filename2)
    prod1=list(file1_prod.keys())
    prod2=list(file2_prod.keys())
    intersection_set=set.intersection(set(prod1),set(prod2))
    intersection=list(intersection_set) 
    intersection.remove('protein') #remove "uncharacterized protein"
    # generate a rondom number
    rand=np.random.randint(0,len(intersection),1)
    rand_product=intersection[int(rand)]
    print("Selected common gene is -- ", rand_product)
    seq1=file1_seq[file1_prod[rand_product]]
    seq2=file2_seq[file2_prod[rand_product]]

    return seq1,seq2
# I will use the files downloaded above,
file1="NC_001133.9.gb"
file2="NC_047487.1.gb"
## 
S1,S2=getcommoncds(file1,file2)

#print("Ok")


Selected common gene is --  tfc3


In [4]:
#Blosum62 matrix 
# BLOSUM62 table from NCBI https://www.ncbi.nlm.nih.gov/IEB/ToolBox/C_DOC/lxr/source/data/BLOSUM62#0001
# I used 23 aminoacid table
def BLOSUMMat(aa1, aa2):
    blosumfile=open("Blosum62_23AA.csv",'r')
    Blosum_Dictionary={}
    line1=blosumfile.readline().strip().split(",")

    for i in line1[1:]:
        Blosum_Dictionary[i]={}
        val=blosumfile.readline().strip().split(",")
        aa=1
        for j in val[1:]:
            Blosum_Dictionary[i][line1[aa]]=j
            aa+=1
    return Blosum_Dictionary[aa1][aa2]
## get alignment score of two aligned sequences using Blosum62
def alignment_score(S1, S2,gap=-8):
    score=0
    for i in range(0,len(S1)):
        if S1[i]=="-" or S2[i]=="-":
            score+=gap
        else:
            score+=int(BLOSUMMat(S2[i],S1[i]))
    return score
print("Ok")


Ok


In [5]:
# Needleman Wunch for two protein sequences S1 and S2
def Needleman_Wunch(S1, S2, gap=-8):
    startt=time.time()
    n=len(S1)+1
    m=len(S2)+1

    score_matrix= np.zeros((m,n),dtype=int)
    alignnment_matrix=np.zeros((m,n), dtype=str)

    #initialize matrices
    for i in range(m):
        score_matrix[i][0]=gap*i
        alignnment_matrix[i][0]="V"
    for j in range(n):
        score_matrix[0][j]=gap*j
        alignnment_matrix[0][j]='H'
    alignnment_matrix[0][0]=0
    #fill the rest of the matrices
    for i in range(1,m):
        for j in range(1, n):
            diagonal=score_matrix[i-1][j-1]+ int(BLOSUMMat(S2[i-1],S1[j-1]))
            previous_row=score_matrix[i-1][j]+gap
            previous_column=score_matrix[i][j-1]+gap
            maxval=max(diagonal,previous_column,previous_row)
            score_matrix[i][j]=maxval
            if maxval==diagonal:
                alignnment_matrix[i][j]="D"
            elif maxval==previous_column:
                alignnment_matrix[i][j]="H"
            else:
                alignnment_matrix[i][j]="V"
            #Additional alignments with same score can also create. 
            # if maxval==previous_row:
            #     alignnment_matrix[i][j]="V"
            # elif maxval==diagonal:
            #     alignnment_matrix[i][j]="D"
            # else:
            #     alignnment_matrix[i][j]="H"
            
    #Backtracing the alignment begins here
    S1_al=""
    S2_al=""
    while not (m==1 and n==1):
        if alignnment_matrix[m-1][n-1]=="D":
            S1_al+=S1[n-2]
            S2_al+=S2[m-2]
            m-=1
            n-=1
        elif alignnment_matrix[m-1][n-1]=="H":
            S1_al+=S1[n-2]
            S2_al+="-"
            n-=1
        else:
            S1_al+="-"
            S2_al+=S2[m-2]
            m-=1
    S1_aligned=S1_al[::-1]   # reverse the backtracked sequences to get the correct sequence
    S2_aligned=S2_al[::-1]
    print(score_matrix)
    print(alignnment_matrix)
    S1_a=[S1_aligned[i:i+50] for i in range(0, len(S1_aligned), 50)] # to display the alignment in avisible way I chopped the sequences to 50 bases
    S2_a=[S2_aligned[i:i+50] for i in range(0, len(S2_aligned), 50)]
    sp=8 #space
    for i in range(0,len(S1_a)):
        if i in[1,2,20,200,2000]: # for indentation purpose
            sp+=1
        print("Seq_1 ", i*50+1,S1_a[i], i*50+50)
        print(" "*sp,"|"*len(S1_a[i]))
        print("Seq_2 ", i*50+1,S2_a[i], i*50+50)
        print()
    endt=time.time()-startt
    print("Elapsed time in seconds ",endt)
    return alignment_score(S1_aligned, S2_aligned)
print("Ok")

Ok


In [6]:
Needleman_Wunch(S1,S2)

[[    0    -8   -16 ... -9264 -9272 -9280]
 [   -8     5    -3 ... -9251 -9259 -9267]
 [  -16    -3     6 ... -9239 -9247 -9255]
 ...
 [-8840 -8827 -8815 ...  4757  4749  4741]
 [-8848 -8835 -8823 ...  4749  4755  4747]
 [-8856 -8843 -8831 ...  4741  4750  4760]]
[['0' 'H' 'H' ... 'H' 'H' 'H']
 ['V' 'D' 'H' ... 'H' 'H' 'H']
 ['V' 'V' 'D' ... 'H' 'H' 'H']
 ...
 ['V' 'V' 'V' ... 'D' 'H' 'H']
 ['V' 'V' 'V' ... 'V' 'D' 'D']
 ['V' 'V' 'V' ... 'V' 'D' 'D']]
Seq_1  1 MVLTIYPDELVQIVSDKIASNKGKITLNQLWDISGKYFDLSDKKVKQFVL 50
         ||||||||||||||||||||||||||||||||||||||||||||||||||
Seq_2  1 M------------------------------------------------- 50

Seq_1  51 SCVILKKDIEVYCDGAITTKNVTDIIGDANHSYSVGITEDSLWTLLTGYT 100
          ||||||||||||||||||||||||||||||||||||||||||||||||||
Seq_2  51 ----LKKDIEVYCDSVITTKNVTNIIDDTSHSYSVGITEDSLWTLLTGYT 100

Seq_1  101 KKESTIGNSAFELLLEVAKSGEKGINTMDLAQVTGQDPRSVTGRIKKINH 150
           ||||||||||||||||||||||||||||||||||||||||||||||||||
Seq_2  101 KKESTIGNSAFELLLEVAKAGEKGIN

4760